In [1]:
import json
import pymupdf
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import unicodedata
from pathlib import Path

from openai import OpenAI
from underthesea import ner

# ──────────────── CẤU HÌNH ────────────────
BOOK_METADATA = {
    "NAM_HOA_KINH": {
        "TITLE": "Nam Hoa Kinh",
        "VOLUME": "",
        "AUTHOR": "Trang Tử",
        "PERIOD": "Chiến Quốc",
        "LANGUAGE": "vi",
        "SOURCE": "thuviensach.vn",
    },
    "TRANG_TU_NAM_HOA_KINH": {
        "TITLE": "Nam Hoa Kinh (Bản học thuật)",
        "VOLUME": "",
        "AUTHOR": "Trang Tử",
        "PERIOD": "Chiến Quốc",
        "LANGUAGE": "vi",
        "SOURCE": "Bản học thuật số hoá",
    },
    "TRANG_TU_NAM_HOA_KINH_2": {
        "TITLE": "Nam Hoa Kinh (Bản dịch)",
        "VOLUME": "",
        "AUTHOR": "Trang Tử",
        "PERIOD": "Chiến Quốc",
        "LANGUAGE": "vi",
        "SOURCE": "Bản dịch hiện đại",
    },
}

BASED_ENTITY_GROUPS = [
    "PER",
    "ORG",
    "LOC",
    "ORG",
    "TME",
    "TITLE",
    "NUM",
]


/home/octoopt/workspace/projects/learn-from-basics/nlp-vietnamese-phd/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
# ──────────────── HÀM TIỆN ÍCH ────────────────

def normalize(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())


def clean_text(text: str, clean_patterns: list[str] = None) -> str:
    """
    Làm sạch văn bản khỏi các ký tự HTML, dấu ngoặc, URL, và mẫu đặc biệt.
    """

    if clean_patterns is None:
        clean_patterns = ["***", "---", "___"]

    # 1. Chuẩn hoá Unicode
    text = normalize(text)

    # 2. Xử lý entity HTML phổ biến
    html_entities = {
        "&quot;": '"',
        "&apos;": "'",
        "&amp;": "&",
        "&lt;": "<",
        "&gt;": ">"
    }
    for k, v in html_entities.items():
        text = text.replace(k, v)

    # 3. Xoá các pattern đặc biệt như "***", "---"
    for pattern in clean_patterns:
        text = text.replace(pattern, "")

    # 4. Xoá nội dung trong ngoặc kép hoặc đơn
    text = re.sub(r'"[^"]*"', '', text)
    text = re.sub(r"'[^']*'", '', text)

    # 5. Xoá các đường link
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # 6. Xoá dấu trang hoặc ký tự đặc biệt (ví dụ từ OCR)
    text = re.sub(r"[\u200b\u200e\u202a\u202c]+", "", text)  # các ký tự ẩn
    text = re.sub(r"\s+", " ", text)  # gom khoảng trắng

    return text.strip()


def clean_page(text: str) -> str:
    """
    Làm sạch từng trang OCR: bỏ header/trailer và nối dòng.
    """
    text = normalize(text)
    # Loại bỏ tiêu đề sách lặp lại theo trang
    text = re.sub(r"TRANG TỬ.*NAM HOA KINH.*", "", text, flags=re.I)
    # Nối dòng không phải đoạn
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)
    return text.strip()


def split_paragraphs(text: str) -> list[str]:
    """
    Tách đoạn bằng cách phát hiện xuống dòng kép.
    """
    return [normalize(p) for p in re.split(r"\n\s*\n", text) if p.strip()]


def split_sentences(
    text: str,
    delimiters=[".", "!", "?", "..."],
    is_clean_text: bool = True
) -> list[str]:
    """
    Tách câu theo các dấu kết thúc câu tiếng Việt.
    """

    if is_clean_text:
        text = clean_text(text)

    sentences = []
    current_sentence = ""
    for char in text:
        current_sentence += char
        if char in delimiters:
            if current_sentence.strip():
                sentences.append(normalize(current_sentence))
            current_sentence = ""

    if current_sentence.strip():
        sentences.append(normalize(current_sentence))

    return sentences

def detect_sections(pages):
    section_pattern = re.compile(
        r"^(PHẦN|CHƯƠNG)\s+[IVXLCDM\d]+\.*\s+.+$", re.MULTILINE
    )
    sections = []
    current = {"name": "Giới thiệu", "pages": []}

    for i, txt in enumerate(pages, 1):
        matches = section_pattern.findall(txt)
        if matches:
            if current["pages"]:
                sections.append(current)
            title = re.findall(section_pattern, txt)[0]
            current = {"name": normalize(title), "pages": [(i, txt)]}
        else:
            current["pages"].append((i, txt))
    sections.append(current)
    return sections


# ──────────────── GHI FILE XML ĐẸP ────────────────
def write_pretty_xml(tree, out_path):
    pretty = minidom.parseString(ET.tostring(tree.getroot(), encoding="utf-8"))
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(pretty.toprettyxml(indent="  "))

In [48]:
def ner_underthesea(text: str) -> list[dict]:
    result = ner(text, deep=True)
    return result 

In [51]:
# ──────────────── HÀM CHÍNH ────────────────
def build_xml_for_book(pdf_path, des="output_xml", code: str = ""):
    """
    Tạo file XML từ PDF, chia theo section → page → câu, giữ nguyên số trang gốc.
    """
    # Bước 1: Xác định ID sách
    book_name = Path(pdf_path).stem.upper().replace("-", "_").replace(" ", "_")
    book_id = book_name if book_name in BOOK_METADATA else f"{book_name}_AUTO"

    # Bước 2: Đọc file PDF
    doc = pymupdf.open(pdf_path)
    pages_text = [clean_page(p.get_text()) for p in doc]

    # Bước 3: Tách các section
    sections = detect_sections(pages_text)

    # Bước 4: Tạo gốc XML
    root = ET.Element("root")
    file_el = ET.SubElement(root, "FILE", ID=book_id)

    # Metadata
    meta_info = BOOK_METADATA.get(book_id, {})
    meta = ET.SubElement(file_el, "meta")
    ET.SubElement(meta, "TITLE").text = meta_info.get("TITLE", book_id.title())
    ET.SubElement(meta, "VOLUME").text = meta_info.get("VOLUME", "")
    ET.SubElement(meta, "AUTHOR").text = meta_info.get("AUTHOR", "Không rõ")
    ET.SubElement(meta, "PERIOD").text = meta_info.get("PERIOD", "Không rõ")
    ET.SubElement(meta, "LANGUAGE").text = meta_info.get("LANGUAGE", "vi")
    ET.SubElement(meta, "SOURCE").text = meta_info.get("SOURCE", "Tự động")

    # Bước 5: Tạo các section
    for sect_id, section in enumerate(sections):
        sect_el = ET.SubElement(
            root, "SECT", ID=f"{code}.{sect_id:03}", NAME=section["name"]
        )

        for page_num, page_text in section["pages"]:
            page_el = ET.SubElement(
                sect_el, "PAGE", ID=f"{code}.{sect_id:03}.{page_num:03}"
            )
            # Split câu theo từng trang gốc
            sentences = split_sentences(page_text)
            for sent_id, sent in enumerate(sentences, 1):
                stc_el = ET.SubElement(
                    page_el,
                    "STC",
                    ID=f"{code}.{sect_id:03}.{page_num:03}.{sent_id:02}",
                )
                stc_el.text = sent

                ner_entity = ner_underthesea(sent)
                valid_entities = [
                    ent for ent in ner_entity
                    if ent.get("entity", "").split("-")[-1] in BASED_ENTITY_GROUPS
                ]

                if valid_entities:
                    ner_el = ET.SubElement(stc_el, "NER")
                    for entity in valid_entities:
                        base_entity = entity["entity"].split("-")[-1]
                        ET.SubElement(
                            ner_el,
                            "ENTITY",
                            TYPE=base_entity,
                            START=str(entity.get("start")),
                            END=str(entity.get("end")),
                        ).text = entity.get("word")
                        

    # Bước 6: Ghi XML
    tree = ET.ElementTree(root)
    write_pretty_xml(tree, des)
    print(f"✅ Xuất file XML: {des}")

In [52]:
pdf_path = "/home/octoopt/workspace/projects/learn-from-basics/the-notes/test/TrangTu_NamHoaKinh_VanAnh/Nam-hoa-kinh.pdf"
code = "PKS_001"
mode = "sentence"
des = f"nam_hoa_kinh_1_{mode}.xml"

build_xml_for_book(pdf_path, des, code)

✅ Xuất file XML: nam_hoa_kinh_1_sentence.xml


In [4]:
import xml.etree.ElementTree as ET


def count_sentences(xml_file_path):
    # Parse the XML file
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Find all STC elements (sentences)
    sentences = root.findall(".//STC")

    # Count the sentences
    total_sentences = len(sentences)

    # Count sentences per section
    sections = {}
    for section in root.findall(".//SECT"):
        section_name = section.get("NAME", "Unknown")
        section_sentences = section.findall(".//STC")
        sections[section_name] = len(section_sentences)

    return total_sentences, sections


# Path to your XML file
xml_file = "/home/octoopt/workspace/projects/learn-from-basics/the-notes/others/Trang-tu-nam-hoa-kinh_PKS_001_sentence.xml"

# Get the counts
total, section_counts = count_sentences(xml_file)

print(f"Total number of sentences: {total}")
print("\nNumber of sentences per section:")
for section, count in section_counts.items():
    print(f"- {section}: {count} sentences")

Total number of sentences: 6760

Number of sentences per section:
- Giới thiệu: 6760 sentences


In [7]:
def get_pdf_metadata(pdf_path):
    """
    Extract metadata from a PDF file using PyMuPDF.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        dict: Dictionary containing PDF metadata
    """
    doc = pymupdf.open(pdf_path)
    print(doc)
    meta = doc.metadata

    # Convert to a dictionary and clean up None values
    metadata = {
        "title": meta.get("title", "").strip() or None,
        "author": meta.get("author", "").strip() or None,
        "subject": meta.get("subject", "").strip() or None,
        "keywords": meta.get("keywords", "").strip() or None,
        "creator": meta.get("creator", "").strip() or None,
        "producer": meta.get("producer", "").strip() or None,
        "creation_date": meta.get("creationDate", "").strip() or None,
        "mod_date": meta.get("modDate", "").strip() or None,
        "format": meta.get("format", "").strip() or None,
        # 'encryption': meta.get('encryption', '\n').strip() or None,
        "page_count": len(doc),
    }

    doc.close()
    return metadata


# Example usage:
pdf_path = "/home/octoopt/workspace/projects/learn-from-basics/nlp-vietnamese-phd/temp/Nam-hoa-kinh.pdf"
metadata = get_pdf_metadata(pdf_path)
print(metadata)

Document('/home/octoopt/workspace/projects/learn-from-basics/nlp-vietnamese-phd/temp/Nam-hoa-kinh.pdf')
{'title': None, 'author': None, 'subject': None, 'keywords': None, 'creator': None, 'producer': 'iLovePDF', 'creation_date': None, 'mod_date': "D:20221109160826+07'00'", 'format': 'PDF 1.4', 'page_count': 139}
